# Lab 2 - Password Bypass

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

In the previous lab, we saw how voltage glitching can be used to cause faults on a target. In the previous lab, we used this to disrupt a calculation. In this lab, we'll look at something a little more realistic. Consider the following code from `linux-util-2.24`:

```C
/*
 *   auth.c -- PAM authorization code, common between chsh and chfn
 *   (c) 2012 by Cody Maloney <cmaloney@theoreticalchaos.com>
 *
 *   this program is free software.  you can redistribute it and
 *   modify it under the terms of the gnu general public license.
 *   there is no warranty.
 *
 */

#include "auth.h"
#include "pamfail.h"

int auth_pam(const char *service_name, uid_t uid, const char *username)
{
    if (uid != 0) {
        pam_handle_t *pamh = NULL;
        struct pam_conv conv = { misc_conv, NULL };
        int retcode;

        retcode = pam_start(service_name, username, &conv, &pamh);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_authenticate(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_acct_mgmt(pamh, 0);
        if (retcode == PAM_NEW_AUTHTOK_REQD)
            retcode =
                pam_chauthtok(pamh, PAM_CHANGE_EXPIRED_AUTHTOK);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        retcode = pam_setcred(pamh, 0);
        if (pam_fail_check(pamh, retcode))
            return FALSE;

        pam_end(pamh, 0);
        /* no need to establish a session; this isn't a
         * session-oriented activity...  */
    }
    return TRUE;
}
```

This code controls [authentication in Linux](https://en.wikipedia.org/wiki/Linux_PAM). You can see it doing a series of checks and, if any of these checks fail, the function returns `FALSE` and the authentication fails. If all these checks pass, then the authentication succeeds. This seems pretty secure, but consider what would happen if we were able to corrupt that `uid != 0` check at the beginning of the code similar to how we corrupted the calculation in the previous lab: we might be able to get the code to jump straight to the end and bypass all those other authentication checks!

We don't be attacking Linux code in this lab, but we will be attacking something similar to the above code - a password check. The password check is in the same `simpleserial-glitch` firmware from the previous lab. This is the code for the password check:

```C
uint8_t password(uint8_t* pw)
{
    char passwd[] = "touch";
    char passok = 1;
    int cnt;

    trigger_high();

    //Simple test - doesn't check for too-long password!
    for(cnt = 0; cnt < 5; cnt++){
        if (pw[cnt] != passwd[cnt]){
            passok = 0;
        }
    }
    
    trigger_low();
    
    simpleserial_put('r', 1, (uint8_t*)&passok);
    return passok;
}
```

Let's connect to the scope and check out this password check:

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_SAM4S'
SS_VER = 'SS_VER_2_1'

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

In [ ]:
fw_path = "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
if SS_VER=="SS_VER_2_1":
    target.reset_comms()
    
def reboot_flush():
    reset_target(scope)
    target.flush()

Let's try to communicate with the target. The password command on `simpleserial-glitch` is `"p"`, takes a 5 byte long password and sends back a 1-byte `"r"` packet. This `"r"` packet will be 0 if the password is incorrect and 1 if it is correct. Try sending a password to the device:

In [ ]:
reboot_flush()

# ###################
# Add your code here to send the password and read the response (Code Block 1)
# ###################
raise NotImplementedError("Add your code here, and delete this.")

Next, let's setup our glitch. We'll make use of the `vglitch_setup()` method to do the setup:

In [ ]:
scope.vglitch_setup('hp', default_setup=False)

Now all we need to do is setup the glitch controller and write another glitch loop. We're not doing calibration of the glitch this time, so there's no need to add a "repeat" parameter.

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None})

Setup your width, offset, and ext_offset ranges and steps. You should use width/offset ranges that worked for you in the last lab. As for ext_offset, it shouldn't be too far beyond the trigger, so something like 0-150 should work.

In [ ]:
# ###################
# Add your code here to set your glitch ranges/steps (Code Block 2)
# ###################
raise NotImplementedError("Add your code here, and delete this.")
        
gc.set_step("ext_offset", 1)

In [ ]:
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.1

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.width = glitch_settings[0]
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.ext_offset = glitch_settings[2]
    
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        gc.add("reset")
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    ret = scope.capture()
    
    if ret:
        gc.add("reset")
        reboot_flush()
    else:
        # ###################
        # Add your code here to read the password response and see if you glitched it (Code Block 3)
        # ###################
        raise NotImplementedError("Add your code here, and delete this.")
                    
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

If you didn't get any glitches, try picking another area for your width/offset ranges.

Let's see where we needed to target for our glitch to work:

In [ ]:
gc.calc(["width", "offset"], "success_rate")

In [ ]:
scope.dis()
target.dis()